In [2]:
import pandas as pd
from top2vec import Top2Vec
from tqdm import tqdm
import nltk
stop=nltk.corpus.stopwords.words('english')
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk import word_tokenize
import os

/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packag

In [31]:
first=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-first-label.csv')
between=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-between-label.csv')
second=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-second-label.csv')

In [2]:
kkk_accessible=pd.read_csv('/Volumes/T7/uni-UIUC/accessible_archives/Post Civil War AFAM Newspapers in the South (Accessible Archives)/kkk-accessible.csv')

In [3]:
def lemmatization(dataframe:pd.DataFrame()):
    dataframe['article']=dataframe['article'].str.lower()
    dataframe['stopword']=dataframe['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]','')
    dataframe['lemma']=dataframe['punct'].apply(lambda row: ' '.join([w.lemma_ for w in nlp(row)]))
    dataframe['token']=dataframe['lemma'].apply(word_tokenize)
    return dataframe

In [35]:
african_between_lemma=lemmatization(between)
#between: 33m 47s

In [ ]:
african_between_lemma.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-between-label-lemma.csv', index=False)

In [33]:
african_second_lemma=lemmatization(second)
#second: 25m 30s

In [34]:
african_second_lemma.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-second-label-lemma.csv', index=False)

In [4]:
accessible_archives_lemma=lemmatization(kkk_accessible)
#81m 49s

In [5]:
accessible_archives_lemma.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-accessible-lemma.csv', index=False)

In [11]:
model_accessible=Top2Vec(documents=accessible_archives_lemma['lemma'].tolist(), speed='learn', workers=4)

2023-11-09 18:17:25,708 - top2vec - INFO - Pre-processing documents for training
2023-11-09 18:18:05,535 - top2vec - INFO - Creating joint document/word embedding
2023-11-09 19:24:27,585 - top2vec - INFO - Creating lower dimension embedding of documents
2023-11-09 19:24:44,771 - top2vec - INFO - Finding dense areas of documents
2023-11-09 19:24:46,205 - top2vec - INFO - Finding topics


In [7]:
model_second=Top2Vec(documents=african_second_lemma['lemma'].tolist(), speed='learn', workers=4)
#second: 12m 21s

2023-10-14 17:16:56,894 - top2vec - INFO - Pre-processing documents for training
2023-10-14 17:17:05,629 - top2vec - INFO - Creating joint document/word embedding
2023-10-14 17:28:33,023 - top2vec - INFO - Creating lower dimension embedding of documents
2023-10-14 17:28:46,428 - top2vec - INFO - Finding dense areas of documents
2023-10-14 17:28:47,770 - top2vec - INFO - Finding topics


In [10]:
model_between=Top2Vec(documents=african_between_lemma['lemma'].tolist(), speed='learn', workers=4)

2023-10-16 11:47:18,966 - top2vec - INFO - Pre-processing documents for training
2023-10-16 11:47:29,753 - top2vec - INFO - Creating joint document/word embedding
2023-10-16 12:04:36,838 - top2vec - INFO - Creating lower dimension embedding of documents
2023-10-16 12:04:54,812 - top2vec - INFO - Finding dense areas of documents
2023-10-16 12:04:56,167 - top2vec - INFO - Finding topics


In [27]:
model_second.save("/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-second-model")


In [11]:
model_between.save("/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-between-model")

In [22]:
model_accessible.save('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-accessible-model')

Load Top2vec model

In [24]:
model = Top2Vec.load("/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-second-model")

In [25]:
topic_sizes, topic_nums = model.get_topic_sizes()

In [26]:
print(len(topic_sizes), len(topic_nums))

281 281


In [27]:
id_dic={}
topic_id={}
for element in zip(topic_nums, topic_sizes):
    documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=element[0], num_docs=element[1])
    for score, id in zip(document_scores, document_ids):
        id_dic[id]=score
        topic_id[id]=element[0]

In [28]:
topic_words, word_scores, topic_scores = model.get_topics(len(topic_sizes))

In [29]:
topic_words

array([['bandit', 'robber', 'robbery', ..., 'ransack', 'steal',
        'messenger'],
       ['creed', 'americanism', 'racial', ..., 'welfare', 'ligious',
        'country'],
       ['stare', 'softly', 'you', ..., 'dear', 'exclaim', 'door'],
       ...,
       ['goose', 'flog', 'fiogge', ..., 'lehi', 'morehouse', 'beaumont'],
       ['sargent', 'gail', 'investment', ..., 'profit', 'worthless',
        'disapproval'],
       ['aileen', 'charming', 'synopsis', ..., 'cn', 'you', 'curiosity']],
      dtype='<U15')

In [30]:
word_scores

array([[0.7088991 , 0.6822487 , 0.63060385, ..., 0.3717241 , 0.37021863,
        0.36933   ],
       [0.51066357, 0.5102603 , 0.48508552, ..., 0.32959738, 0.3281705 ,
        0.32627568],
       [0.5714329 , 0.5132176 , 0.50531596, ..., 0.33820325, 0.33773383,
        0.337372  ],
       ...,
       [0.60776526, 0.36422482, 0.34146157, ..., 0.20897074, 0.20857148,
        0.2071973 ],
       [0.63310593, 0.47687355, 0.44459054, ..., 0.21232896, 0.20788704,
        0.20214349],
       [0.78701687, 0.5215063 , 0.5136021 , ..., 0.1853756 , 0.18326403,
        0.18190877]], dtype=float32)

In [31]:
df_words=pd.DataFrame(topic_words).transpose()
df_words.columns=topic_nums
df_words.columns = df_words.columns.astype(str)

In [32]:
df_scores=pd.DataFrame(word_scores).transpose()
df_scores.columns=topic_nums
df_scores.columns=df_scores.columns.astype(str)

In [33]:
df_words.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/second-topic-words.csv', index=False)

In [34]:
df_scores.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/second-topic-scores.csv', index=False)

In [18]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["wizard"], num_topics=5)
topic_words

[array(['sore', 'canker', 'cold', 'loosen', 'soothe', 'earache', 'heal',
        'toothache', 'gargle', 'constipation', 'neck', 'chest', 'troubled',
        'rub', 'tired', 'quick', 'generous', 'sprain', 'often', 'bottle',
        'relief', 'sick', 'headache', 'medicine', 'hamiin', 'druggist',
        'liver', 'bruise', 'pill', 'aration', 'shelf', 'throat', 'prep',
        'ache', 'pink', 'simple', 'pleasant', 'guarantee', 'burn',
        'reliable', 'satisfied', 'hamlin', 'stiff', 'dependable', 'stings',
        'bite', 'occur', 'deep', 'constipate', 'healing'], dtype='<U15'),
 array(['soothe', 'penetrate', 'heal', 'constipation', 'reliable',
        'generous', 'troubled', 'sprain', 'sick', 'stings', 'sting',
        'druggist', 'bite', 'sore', 'headache', 'soreness', 'ache',
        'stiff', 'medicine', 'bruise', 'liver', 'hamiin', 'canker',
        'bottle', 'pill', 'sooth', 'earache', 'limber', 'absolutely',
        'pleasant', 'antiseptic', 'pink', 'toothache', 'guarantee',
     

In [20]:
print(topic_nums)
print(topic_scores)

[ 72 221  92  21 128]
[0.26426785 0.25976787 0.25188837 0.23514197 0.21826114]


In [23]:
wizard_relevant_index=[]
wizard_relevant_score=[]
wizard_relevant_lemma=[]
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=21, num_docs=411)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    wizard_relevant_index.append(doc_id)
    wizard_relevant_score.append(score)
    wizard_relevant_lemma.append(doc)

In [36]:
pd.DataFrame.from_dict({'index':wizard_relevant_index, 'score':wizard_relevant_score, 'lemma':wizard_relevant_lemma}).to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-wizard-relevant-df.csv', index=False)

In [32]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["lynch"], num_docs=second.shape[0])
lynch_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
lynch_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-lynch-topic.csv')

In [33]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["riot"], num_docs=second.shape[0])
riot_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
riot_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-riot-topic.csv')

In [34]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["mob"], num_docs=second.shape[0])
mob_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
mob_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-mob-topic.csv')

In [35]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["wizard"], num_docs=second.shape[0])
wizard_df=pd.DataFrame({'doc':documents, 'id':document_ids, 'score':document_scores})
wizard_df.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-wizard-topic.csv')

In [13]:
model = Top2Vec.load("/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-between-model")

In [14]:
topic_sizes, topic_nums = model.get_topic_sizes()

In [15]:
print(len(topic_sizes), len(topic_nums))

293 293


In [16]:
id_dic={}
topic_id={}
for element in zip(topic_nums, topic_sizes):
    documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=element[0], num_docs=element[1])
    for score, id in zip(document_scores, document_ids):
        id_dic[id]=score
        topic_id[id]=element[0]

In [17]:
topic_words, word_scores, topic_scores = model.get_topics(len(topic_sizes))

In [18]:
topic_words

array([['whisper', 'you', 'smile', ..., 'but', 'cry', 'aloud'],
       ['fatally', 'injure', 'suicide', ..., 'states', 'apoplexy',
        'sentence'],
       ['holdup', 'footpad', 'saloon', ..., 'sidewalk', 'rifie',
        'robbed'],
       ...,
       ['hamlin', 'soreness', 'sore', ..., 'prevent', 'invariably',
        'disappoint'],
       ['concentrate', 'hamlin', 'oil', ..., 'heal', 'substitute',
        'syrup'],
       ['mishap', 'hamlin', 'inns', ..., 'disease', 'symptom', 'monia']],
      dtype='<U14')

In [19]:
df_words=pd.DataFrame(topic_words).transpose()
df_words.columns=topic_nums
df_words.columns = df_words.columns.astype(str)

In [20]:
df_scores=pd.DataFrame(word_scores).transpose()
df_scores.columns=topic_nums
df_scores.columns=df_scores.columns.astype(str)

In [23]:
df_words.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/between-topic-words.csv', index=False)

In [22]:
df_scores.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/between-topic-scores.csv', index=False)

In [20]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["wizard"], num_topics=40)
topic_words

[array(['hamlin', 'pain', 'oil', 'neuralgia', 'inflammation', 'soreness',
        'rheumatism', 'cure', 'remedy', 'ache', 'wizard', 'sore',
        'painful', 'ailment', 'druggist', 'bottle', 'surely', 'allay',
        'suffer', 'suffering', 'earache', 'soothe', 'relief', 'weakness',
        'lame', 'neu', 'misery', 'lung', 'headache', 'heal', 'throat',
        'symptom', 'honest', 'disease', 'stiffness', 'wrapper', 'use',
        'toothache', 'sprain', 'kidney', 'daniels', 'rub', 'sufferer',
        'moth', 'lief', 'stomach', 'consumption', 'cold', 'stiff',
        'seldom'], dtype='<U14'),
 array(['cure', 'hamlin', 'neuralgia', 'oil', 'pain', 'remedy',
        'rheumatism', 'inflammation', 'bottle', 'ache', 'wizard',
        'disease', 'druggist', 'soreness', 'medicine', 'ailment', 'kidney',
        'sore', 'soothe', 'headache', 'earache', 'relief', 'lung',
        'obstinate', 'cough', 'painful', 'suffer', 'cold', 'syrup',
        'afflict', 'allay', 'chronic', 'stomach', 'symptom',

In [28]:
between=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/kkk-between-label.csv')
between[between['article'].str.contains('wizard')]

,index,headline,newspaper_name,date,article,sn,label
12,6_1872-02-10_p1_sn84026820_00271765277_1872021...,NaN,Middletown transcript.,1872-02-10,Miuuit. in n plain but reliable wagon.\nwith n...,sn84020422,neither
16,2_1872-10-08_p7_sn83030313_00271743804_1872100...,JEROME PARK RACES.\n\nThird Day OF tho Fall Me...,The New York herald.,1872-10-08,Third Day OF tho Fall Meeting-The'\n\n\nEvents...,sn89053204,neither
30,22_1872-12-28_p1_sn84026820_00271765277_187212...,NaN,Middletown transcript.,1872-12-28,And Winter with his aged locks--and breathe\nl...,sn84020558,neither
42,5_1872-11-30_p2_sn83021205_00517171876_1872113...,SATAN in Esuaale-A swiss Couple the Vic\n\n\ns...,Worcester daily spy.,1872-11-30,"\n\n\nFrom the Buffalo Courier\n\n\n\ genuine,...",sn82014086,neither
50,4_1872-11-04_p6_sn83030214_00206531125_1872110...,Medical AND SURGICAL NOTES.,New-York tribune.,1872-11-04,"c,,.srrr,rrp FOR CuIORoFORM-Dr. Richard\n~ has...",sn83045462,neither
...,...,...,...,...,...,...,...
44481,18_1914-12-15_p5_sn88051110_00414213182_191412...,Swell Headedness Is Not Self-Confdence,The Spectator.,1914-12-15,wizard fault In this particular he devoted him...,sn96076588,aa
44483,5_1914-02-11_p6_sn84022472_00295867267_1914021...,"THOMAS A. EDISON, 6z TODAY, THINKS MAN MAY LIV...",The Bridgeport evening farmer.,1914-02-11,5\n\n That men may attain to the age Of 150 ye...,sn95060626,aa
44484,1_1914-03-02_p9_sn84022472_00295867279_1914030...,THE PLAZA,The Bridgeport evening farmer.,1914-03-02,The name of Gus Edwards has al- ways been syno...,sn91055359,aa
44485,8_1914-03-08_p5_sn89051285_00414212761_1914030...,AT THE PRINCESS.,The Sentinel=record.,1914-03-08,Here is another big musical. spec tacle and va...,sn95060626,aa


In [29]:
between.loc[44481, 'article']

'wizard fault In this particular he devoted himself fully seeing that the did not develop this fault and his assduy rewarded the story goe when the became of his valued aids.\n\n Therein lies large lesson starting life All of the overofciousn of indicating tenesspro- cees exaggerated the vacu "we headedness These little might become big in time if the abnormal growth did their other growth.\n\n We this in public oe in private employ, social life everywhere There of the disease One of the most prevalent the arrogang ones self of hghsoundng esusua woud-be hereofand there many other manifestations.\n\n excellent swell hadednes seonfdence. Rather bumpousness and the world quick and keen precate undong of these who unconscousy riding for fa'

In [21]:
print(topic_nums)
print(topic_scores)

[271 242 142 243  63 198 270 260 207 221 233 253 163 140 280 224 188 269
 230 158 181 121 249 246 273 267 265 235 248 240 241 282  90 286 272 228
 245 278 289 234]
[0.42263796 0.40789729 0.38109911 0.37943853 0.36624621 0.33884976
 0.33695302 0.3281438  0.32686744 0.32414584 0.32234333 0.32003527
 0.31390767 0.31324483 0.30688628 0.30600683 0.29760094 0.2958992
 0.29568734 0.29338197 0.29239106 0.2899169  0.2867508  0.28413682
 0.28226441 0.28029941 0.27750024 0.27285904 0.27234614 0.272297
 0.2707148  0.26926034 0.26814044 0.26649017 0.2657422  0.26275471
 0.25931561 0.25651576 0.25604811 0.25501709]


In [50]:
second_topic_words=pd.read_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/second-topic-words.csv')

In [53]:
second_topic_words=second_topic_words.drop(columns=['Unnamed: 0'])

In [54]:
second_topic_words

,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
0,americanism,bandit,stare,fiery,cashier,electrical,bastrop,pouch,mop,mcadoo,...,frye,fern,rathenau,juanita,gent,strike,sargent,kennel,revenue,aileen
1,creed,robber,she,ceremony,bandit,inventor,morehouse,bandit,csc,adoo,...,henson,costume,reichstag,dawn,abrupt,avert,gail,terri,internal,crane
2,citizenship,holdup,softly,parade,robber,invention,watt,train,enamel,convention,...,ringer,dance,chancellor,luke,female,railway,boyd,dog,income,synopsis
3,religion,revolver,you,regalia,bank,edison,rouge,mail,houseware,plank,...,horseshoe,enjoyable,germany,ye,startle,overwhelmingly,vestry,brook,tax,charming
4,racial,robbery,suddenly,ceremonial,vault,steinmetz,fourche,robber,saucepan,delegate,...,saunder,decoration,assassination,shudder,occupant,wage,disgrace,breed,vault,porcelain
5,religious,jewelry,eye,procession,cash,inven,parish,passenger,quart,dele,...,tourna,halloween,minister,thar,depot,striker,investment,bulldog,cashier,dwight
6,intolerance,rob,glance,initiation,currency,menlo,coco,loot,aluminium,bryan,...,tournament,pumpkin,berlin,reckon,snap,subway,nicholas,sporting,currency,masque
7,fundamental,cash,then,cross,rob,electricity,mckoin,holdup,enameled,platform,...,percentage,refreshment,assassin,vey,suitcase,elevate,shepherd,thoroughbred,stafford,scar
8,hatred,pistol,murmur,naturalization,robbery,trical,skipwith,robbery,galvanize,presidency,...,pionship,decorate,assassinate,heap,parlor,braddock,jerk,winner,cash,gluck
9,bigotry,police,gasp,klansmen,bookkeeper,experiment,daniel,baggage,each,deadlock,...,farrell,masquerade,ministry,abruptly,uncertain,annually,allison,own,taxis,klein


In [48]:
topic_words, word_scores, topic_nums = model.get_topics

array([2000, 1582, 1572, 1493, 1397, 1170,  931,  789,  700,  679,  656,
        648,  584,  577,  560,  557,  552,  516,  508,  505,  486,  477,
        469,  448,  425,  414,  403,  389,  385,  373,  370,  361,  349,
        324,  308,  303,  300,  299,  286,  285,  282,  281,  277,  271,
        269,  265,  253,  253,  250,  242,  233,  232,  228,  220,  214,
        214,  212,  210,  206,  198,  196,  196,  196,  186,  168,  167,
        165,  165,  157,  157,  152,  147,  147,  142,  141,  137,  137,
        136,  133,  131,  130,  130,  127,  127,  127,  123,  122,  121,
        120,  120,  113,  113,  112,  108,  106,  106,  104,  104,  104,
        103,  101,   99,   99,   98,   97,   96,   95,   95,   95,   94,
         91,   91,   90,   89,   89,   87,   86,   83,   82,   82,   82,
         80,   77,   77,   76,   75,   74,   72,   71,   70,   70,   70,
         68,   67,   67,   65,   65,   65,   64,   63,   63,   63,   61,
         61,   60,   60,   59,   59,   59,   58,   

In [3]:
model = Top2Vec.load("/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/kkk-accessible-model")

In [4]:
topic_sizes, topic_nums = model.get_topic_sizes()

In [5]:
print(len(topic_sizes), len(topic_nums))

357 357


In [6]:
topic_words, word_scores, topic_nums = model.get_topics(len(topic_nums))

In [81]:
id_dic={}
topic_id={}
for element in zip(topic_nums, topic_sizes):
    documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=element[0], num_docs=element[1])
    for score, id in zip(document_scores, document_ids):
        id_dic[id]=score
        topic_id[id]=element[0]

In [83]:
df_words=pd.DataFrame(topic_words).transpose()
df_words.columns=topic_nums
df_words.columns = df_words.columns.astype(str)

In [84]:
df_words.to_csv('/Volumes/T7/chroniclingamerica/american-stories/keywords/top2vec/accessible-archives-topic-words.csv')

In [61]:
cols_with_ku_in_row 

Index([], dtype='object')

In [7]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["mob"], num_topics=40)
topic_words[0:5]

[array(['jail', 'assault', 'murder', 'escape', 'arrest', 'allege',
        'wasfound', 'themurder', 'shoot', 'assailant', 'apprehend',
        'thejail', 'mob', 'thecrime', 'jury', 'themob', 'guilty',
        'policeman', 'prisoner', 'wasarreste', 'chargedwith', 'police',
        'victim', 'custody', 'shot', 'theprisoner', 'deputysheriff',
        'criminally', 'detective', 'acharge', 'injail', 'commit',
        'convict', 'arrestedand', 'criminal', 'implicate', 'crime',
        'tragedy', 'inquest', 'grandjury', 'thejury', 'arrested',
        'occurred', 'accuse', 'identify', 'attemptedto', 'posse',
        'wasshot', 'countyjail', 'coroner'], dtype='<U15'),
 array(['thebrownsville', 'affray', 'negrosoldier', 'senatorforaker',
        'foraker', 'theshoote', 'shooting', 'penrose', 'martial',
        'innocence', 'brownsville', 'senatecommittee', 'guilt',
        'battalion', 'infantry', 'shoot', 'thesoldier', 'thecharge',
        'implicate', 'investigation', 'testify', 'accuse', 'bar

In [35]:
topic_nums[0:5]

array([121,   0,  12,   4,  47])

In [41]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=121, num_docs=20)

In [42]:
documents

array(['fifth annualmeete ofnational independent politicalleague . hold inphiladelphia wednesday , thursday friday , july3rd , 4th 5th , 1912.at boston meeting n.i . p. l hold last august , wasvote bold next annual meeting(the fifth ) either harper\'sferry martinsburg , w. va . ithas since find impracticableand inexpedient hold nextmeeting league either oneof place and , majority ofthe member executive committeehaving indicate choiceof philadelphia place andwednesday , thursday friday , july 3rd , 4th 5th , 1912 , thetime hold fifthannual session league , byvirtue power vest aspresident n. i. p. l , i , j. r.clifford , hereby give notice thatthe fifth annual meeting saidleague hold zion baptistchurch , n. 13th st . , wallace , philadelphia , pa . , july 3rd , 4thand 5th , a. d. 1912 , respectfullyrequest urge themembers executive committeeand president localleague state territorytake step select delegatesto annual meeting . also requestthat member say committeeand president otheroffice

In [44]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=12, num_docs=20)

In [45]:
documents

array(['ratification meeting natchitoches.the lead 2nd influential member ofthe republican party differentward meet evening nineteenth , at hall central club , forthe purpose ascertain true historyof late convention hold city ofnew orleans , august 9 , 1871.the meeting call order byjoseph austin , president , proceedingsof turner hall convention andthat customhouse read byjohn g. lewis , secretary , large conconcoursof republicans present.hon . raford blunt sad h. raby , delegate;to late convention hold thecity new orleans , august 9 , 1871 , addressedthe meeting length relative tothe action two convention , presentinga true history atrocitiescommitte lowell - packard boltersand disorganizer . close thespeeche , follow resolution wereread present club : resolve , we , republican partyof parish natchitoches , sustainand recognize action legaldelegate represent republicanparty parish natchitoche , wit : hon . raford blunt , henry baby andm . p. blackstone , we , republicanparty , tender 

In [44]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument 
tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate(data)]
model = Doc2Vec(vector_size=300, min_count=1, epochs=50)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

ishmaei strike across mouth , exclaim : silent , accurse wizard . shall howl loud yo1r ghost - dog . " |i meant harm " answer man humbly . curious gleam eye . command chief ? " watch here . think daughter shouter . call inkosazana - y - zoola . com ing , may need help . bring thirty man bade messenger aye . ibubesi , hidden bush . go summon they , though think mighty inko- sazana . command zulu impis spirit dead , need little help we . '
0.5332522


In [36]:
import umap
from sklearn.datasets import load_digits

# digits = load_digits()

embedding = umap.UMAP().fit_transform(document_vectors)

NameError: name 'document_vectors' is not defined